# 課題 1.2　ロジスティック回帰分析

ロジスティック回帰のインスタンス作成時のオプション指定：
- C=10000.0 を指定すること `LogisticRegression(C=10000.0)`
- 必要に応じて max_iter も指定すること `LogisticRegression(C=10000.0, max_iter=100000)`

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))
# 欠損値 NaN がある行の除去
autos = autos.dropna()
# 確認
autos

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
5,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
7,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
9,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,four,108,mpfi,3.50,2.80,8.8,101.0,5800.0,23,29,16430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
200,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
201,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
202,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


<hr>

### 1.2.1 市街地での燃費の良し悪しと馬力との関係

autos から city_mpg, highway_mpg 列を除外したうえで、city_mpgから中央値よりも大きければ1、小さいか等しければ0を値としたcity_mpg_01列を追加した pandas DataFrame を作成しなさい。

そのDataFrameについて、X軸を horsepower , Y軸を height として、city_mpg_01で色分けをした散布図を作成しなさい。

1. autos から city_mpg, highway_mpg 列を除外した DataFrame

In [2]:
df = autos.drop(['city_mpg', 'highway_mpg'], axis=1)

2. city_mpgから中央値よりも大きければ1、小さいか等しければ0を値としたcity_mpg_01列を追加

In [3]:
# city_mpgの中央値
city_mpg_median = autos.city_mpg.median()

# city_mpgから中央値よりも大きければ1、小さいか等しければ0を値としたcity_mpg_01列を追加
# - city_mpg 列の各値について、
df['city_mpg_01'] = autos['city_mpg'].map(lambda x: 1 if x > city_mpg_median else 0)

DataFrame の表示

In [4]:
df

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,price,city_mpg_01
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,13950.0,0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,17450.0,0
5,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,17710.0,0
7,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140.0,5500.0,23875.0,0
9,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,four,108,mpfi,3.50,2.80,8.8,101.0,5800.0,16430.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,16845.0,0
200,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19045.0,0
201,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,21485.0,0
202,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,22470.0,0


補足：city_mpg_01の正負の数

In [ ]:
df.groupby('city_mpg_01').size()

X軸を horsepower , Y軸を height として、city_mpg_01で色分けをした散布図

In [ ]:
sns.scatterplot(x=df.horsepower, y=df.height, hue=df.city_mpg_01)

<hr>

### 1.2.2 馬力からの燃費の良し悪しの予測

horsepower を説明変数, city_mpg_01 を目的変数とするロジスティック回帰モデルを作成し、AUC値を求め、また回帰係数 β_0 (切片), β_1 (傾き) を表示しなさい。<br>
AUC, 回帰係数ともに、小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。

1. 用いる関数の定義

In [ ]:
# 散布図にシグモイド曲線を重ねて表示
def sigmoid(x, b0, b1):
    return 1/(1 + np.exp(-(b0 + b1 * x)))

def Logistic_Model(X, Y):
    """
    ロジスティック回帰モデルを作成
    """
    model = LogisticRegression(C=10000.0, max_iter=10000000)
    model.fit(X, Y)
    return model

def Logistic_Plot(model, X, Y):
    # 散布図
    sns.scatterplot(x=X.iloc[:,0], y=Y)

    # パラメータ
    b0 = model.intercept_[0]
    b1 = model.coef_[0][0]

    # y=0.5となる分割線
    boundary = -b0 / b1
    plt.plot([boundary, boundary], [0, 1], color='lightblue')

    # シグモイド曲線
    _max = X.iloc[:,0].max()
    _min = X.iloc[:,0].min()
    _step = (_max - _min) / 100
    _X = np.arange(_min, _max + _step, _step)
    plt.plot(_X, sigmoid(_X, b0, b1), color='brown')
    
    return boundary

def calc_AUC(model, X, Y):
    """
    AUCの算出
    """
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    return auc(fpr, tpr)

from sklearn.metrics import roc_auc_score
def calc_AUC2(model, X, Y):
    """
    AUCの算出: roc_auc_score を使用
    """
    return roc_auc_score(Y, model.decision_function(X))

2. horsepower を説明変数, city_mpg_01 を目的変数とするロジスティック回帰モデルを作成

In [ ]:
# 説明変数、目的変数を作成
X = df[['horsepower']]
Y = df.city_mpg_01

model = Logistic_Model(X, Y)

AUC値を求め、また回帰係数 β_0 (切片), β_1 (傾き) を表示

In [ ]:
AUC = calc_AUC(model, X, Y)
AUC2 = calc_AUC2(model, X, Y)
print('AUC={:.3f} {:.3f}'.format(AUC, AUC2))

b0 = model.intercept_[0]
b1 = model.coef_[0][0]
print('b0={:.3f}, b1={:.3f}'.format(b0, b1))

補足：モデルのシグモイド曲線

In [ ]:
boundary = Logistic_Plot(model, X, Y)
print('Boundary={:.0f}'.format(boundary))

<hr>

### 1.2.3 燃費の良し悪しと他の原因との関係

horsepower, curb_weight, aspiration を説明変数、city_mpg_01 を目的変数とするロジスティック回帰モデルを作成し、
1. AUC値を求めなさい。<br>
小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。
2. aspiration が turbo かそうではないかのそれぞれについて、curb_weight は中央値に固定したうえで、X軸に horsepower をとり最小値から最大値まで変化させ、Y軸に city_mpg_01=1 になる確率（シグモイド曲線）をグラフで描画しなさい。
  - グラフが turbo かそうではないかわかるように、色分けして表示すること。
  - できれば legend を付けること。付けられなければ、コメントやMarkdownのセルに文章を記載し補足すること。
3. 結果を簡単に考察しなさい。

horsepower, curb_weight, aspiration を説明変数、city_mpg_01 を目的変数とするロジスティック回帰モデル

In [ ]:
# 説明変数、目的変数を作成
X = df[['horsepower', 'curb_weight', 'aspiration']]
X = pd.get_dummies(data=X, drop_first=True)
Y = df.city_mpg_01

# ダミー変数化された後の説明変数を確認
# - aspiration_turbo: turbo の時に 1
X

In [ ]:
# ロジスティック回帰モデルを作成
model = Logistic_Model(X, Y)

1. AUC値を求めなさい

In [ ]:
print('AUC={:.3f}'.format(calc_AUC(model, X, Y)))

2. aspiration が turbo かそうではないかのそれぞれについてグラフで描画

In [ ]:
# curb_weight は中央値に固定
cw_median= X.curb_weight.median()

# X軸に horsepower をとり最小値から最大値まで変化
hp_min = X.horsepower.min()
hp_max = X.horsepower.max()
hp_step = (hp_max - hp_min) / 100
hp_X = np.arange(hp_min, hp_max + hp_step, hp_step)

In [ ]:
b0 = model.intercept_[0]
b1 = model.coef_[0][0]
print('b0={:.3f}, b1={:.3f}'.format(b0, b1))

In [ ]:
# b0: 切片
model.intercept_

In [ ]:
# 回帰係数
model.coef_

In [ ]:
# 各係数を取得できることを確認
b0 = model.intercept_[0]
b1 = model.coef_[0][0]
b2 = model.coef_[0][1]
b3 = model.coef_[0][2]
print('b0={:.3f}, b1={:.3f}, b2={:.3f}, b3={:.3f}'.format(b0, b1, b2, b3))

In [ ]:
# シグモイド曲線
# - 説明変数：'horsepower', 'curb_weight', 'aspiration'
def sigmoid_hp(model, hp, cw, turbo):
    b0 = model.intercept_[0]
    b1 = model.coef_[0][0]
    b2 = model.coef_[0][1]
    b3 = model.coef_[0][2]
    return 1/(1 + np.exp(-(b0 + b1 * hp + b2 * cw + b3 * turbo)))

# 曲線の描画
# - horsepower: hp_X = horsepower の最小値から最大値まで変化させた値
# - cw: cw_median = curb_weight は中央値に固定
# - turbo: 0 = turbo ではない
plt.plot(hp_X, sigmoid_hp(model, hp_X, cw_median, 0), color='brown')

In [ ]:
# 曲線の描画
# - horsepower: hp_X = horsepower の最小値から最大値まで変化させた値
# - cw: cw_median = curb_weight は中央値に固定
# - turbo: 1 = turbo ではない
plt.plot(hp_X, sigmoid_hp(model, hp_X, cw_median, 1), color='blue')

In [ ]:
# 重ねて描画
plt.plot(hp_X, sigmoid_hp(model, hp_X, cw_median, 0), color='brown', label='turbo=0')
plt.plot(hp_X, sigmoid_hp(model, hp_X, cw_median, 1), color='blue', label='turbo=1')

# X, Y軸のラベル、レジェンドの表示
plt.xlabel('horsepower')
plt.ylabel('city_mpg_01')
plt.legend()

2. aspiration が turbo かそうではないかのそれぞれについてグラフで描画（別解）

predict_proba(): 予測された確率を計算するメソッド
- 引数: n次元配列（複数の説明変数の列）
- 戻り値: 2次元配列（各列に対応した 0: y=0 となる確率, 1: y=1 となる確率

In [ ]:
# 引数とする説明変数を用意
# - horsepower: hp_X = horsepower の最小値から最大値まで変化させた値
# - cw: cw_median = curb_weight は中央値に固定
# - aspiration_turbo: 0 = turbo ではない
df_plot_turbo_0 = pd.DataFrame(hp_X, columns=['horsepower'])
df_plot_turbo_0['curb_weight'] = cw_median
df_plot_turbo_0['aspiration_turbo'] = 0

# 確認
df_plot_turbo_0

In [ ]:
plt.plot(hp_X, model.predict_proba(df_plot_turbo_0)[:, 1], color='brown')

In [ ]:
df_plot_turbo_1 = pd.DataFrame(hp_X, columns=['horsepower'])
df_plot_turbo_1['curb_weight'] = cw_median
df_plot_turbo_1['aspiration_turbo'] = 1

# 確認
df_plot_turbo_1

In [ ]:
plt.plot(hp_X, model.predict_proba(df_plot_turbo_1)[:, 1], color='blue')

In [ ]:
# 重ねて描画
plt.plot(hp_X, model.predict_proba(df_plot_turbo_0)[:, 1], color='brown', label='turbo=0')
plt.plot(hp_X, model.predict_proba(df_plot_turbo_1)[:, 1], color='blue', label='turbo=1')

# X, Y軸のラベル、レジェンドの表示
plt.xlabel('horsepower')
plt.ylabel('city_mpg_01')
plt.legend()

3. 結果を簡単に考察しなさい。

AUC, 上記グラフからわかることについての考察。

- ダウンサイジングターボ<br>
https://ja.wikipedia.org/wiki/ダウンサイジングコンセプト